# Full outcome model

In [1]:
# import required modules
import numpy as np
import pandas as pd
from math import sqrt
from scipy import stats

from classes.pathway import SSNAP_Pathway
from classes.clinical_outcome import Clinical_outcome

In [2]:
# # TEMPORARY 
# import warnings
# warnings.filterwarnings('ignore')

In [3]:
hospital_performance = pd.read_csv('data/hospital_performance_thrombectomy.csv')

In [4]:
mrs_dists = pd.read_csv('data/mrs_dist_probs_cumsum.csv', index_col='Stroke type')

In [5]:
mrs_dists

,0,1,2,3,4,5,6
Stroke type,,,,,,,
pre_stroke_nlvo,0.582881,0.745419,0.848859,0.951082,0.993055,1.000000,1.0
pre_stroke_nlvo_ivt_deaths,0.576469,0.737219,0.839522,0.940620,0.982131,0.989000,1.0
pre_stroke_lvo,0.417894,0.560853,0.679283,0.843494,0.957269,1.000000,1.0
pre_stroke_lvo_ivt_deaths,0.403644,0.541728,0.656119,0.814731,0.924626,0.965900,1.0
pre_stroke_lvo_mt_deaths,0.402850,0.540662,0.654829,0.813128,0.922807,0.964000,1.0
no_treatment_nlvo,0.197144,0.460000,0.580032,0.707768,0.855677,0.917702,1.0
no_effect_nlvo_ivt_deaths,0.197271,0.460000,0.577583,0.702252,0.845244,0.904454,1.0
t0_treatment_nlvo_ivt,0.429808,0.630000,0.738212,0.848427,0.929188,0.956300,1.0
no_treatment_lvo,0.050000,0.129000,0.265000,0.429000,0.676000,0.811000,1.0


In [6]:
hospital_performance

,stroke_team,admissions,proportion_of_all_with_ivt,proportion_of_all_with_mt,proportion_of_mt_with_ivt,proportion1_of_all_with_onset_known_ivt,proportion2_of_mask1_with_onset_to_arrival_on_time_ivt,proportion3_of_mask2_with_arrival_to_scan_on_time_ivt,proportion4_of_mask3_with_onset_to_scan_on_time_ivt,proportion5_of_mask4_with_enough_time_to_treat_ivt,...,proportion3_of_mask2_with_arrival_to_scan_on_time_mt,proportion4_of_mask3_with_onset_to_scan_on_time_mt,proportion5_of_mask4_with_enough_time_to_treat_mt,proportion6_of_mask5_with_treated_mt,lognorm_mu_onset_arrival_mins_mt,lognorm_sigma_onset_arrival_mins_mt,lognorm_mu_arrival_scan_arrival_mins_mt,lognorm_sigma_arrival_scan_arrival_mins_mt,lognorm_mu_scan_puncture_mins_mt,lognorm_sigma_scan_puncture_mins_mt
0,Addenbrooke's Hospital,602.166667,0.149184,0.026571,0.520833,0.590645,0.679007,0.948930,0.845818,1.0,...,0.980856,0.935706,0.994479,0.046885,4.849537,0.675364,3.804003,0.820081,4.323977,1.264915
1,Basildon University Hospital,486.500000,0.132237,0.011990,0.600000,0.648167,0.572410,0.958449,0.888247,1.0,...,0.977957,0.921863,0.991035,0.023849,4.849547,0.691845,2.820170,1.651319,5.344571,0.673775
2,Blackpool Victoria Hospital,485.833333,0.091938,0.012350,0.444444,0.448370,0.693191,0.941501,0.888628,1.0,...,0.975023,0.917457,0.988625,0.024059,4.694959,0.741009,3.806109,0.959561,4.640248,0.976944
3,Broomfield Hospital,452.166667,0.104681,0.006635,0.555556,0.558791,0.605541,0.991285,0.915385,1.0,...,0.997382,0.949256,0.993548,0.014842,4.878630,0.683373,3.049651,1.028392,5.188837,0.937691
4,Calderdale Royal Hospital,634.666667,0.135504,0.006040,0.739130,0.546218,0.645673,0.946389,0.929976,1.0,...,0.965244,0.931143,0.988467,0.015100,4.735035,0.747534,3.023825,1.283449,5.260841,0.688688
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78,Wirral Arrowe Park Hospital,577.000000,0.093588,0.006355,0.818182,0.740035,0.519906,0.969219,0.895430,1.0,...,0.986747,0.946276,0.990968,0.013672,4.782984,0.734571,3.593126,0.790612,5.259824,0.306376
79,Worthing Hospital,477.000000,0.131726,0.004892,0.642857,0.638365,0.677614,0.953150,0.884746,1.0,...,0.979375,0.926630,0.991935,0.008869,4.780181,0.676788,3.396003,1.152219,5.025557,0.685536
80,Wycombe General Hospital,619.166667,0.134859,0.014266,0.735849,0.723822,0.561547,0.975497,0.924644,1.0,...,0.984375,0.964021,0.987925,0.029444,4.857380,0.679072,2.522393,1.400003,4.947447,0.909323
81,Yeovil District Hospital,310.000000,0.152151,0.009677,0.555556,0.665591,0.624394,0.972833,0.908245,1.0,...,0.983420,0.947313,0.987764,0.019144,4.916434,0.643097,3.256842,0.999543,5.630844,0.495198


## Run trials

In [7]:

# Set general model parameters
scenario_counter = 0
trials = 1

# Iterate through hospitals
for hospital in hospital_performance.iterrows():
    scenario_counter += 1
    print(f'Scenario {scenario_counter}', end='\r')

    # Get data for one hospital
    hospital_name = hospital[0]
    hospital_data = hospital[1]
    run_data = hospital_data
    calibration = 1.0

    patients_per_run = int(run_data['admissions'])

    for trial in range(trials):

        patient_array = SSNAP_Pathway(hospital_name, hospital_data)
        patient_array.run_trial()

    print(stop, here, please)

NameError: name 'stop' is not defined

In [ ]:
patient_array.results_dataframe.head(5).T

In [ ]:
print(stop, here, please)

In [13]:

# Set general model parameters
scenario_counter = 0
trials = 100

# Set up dataframes.

# Record these measures...
outcome_results_columns = [
    'Thrombolysis_rate_(%)',
    'LVO_IVT_mean_shift',
    'LVO_MT_mean_shift',
    'nLVO_IVT_mean_shift',
]

# ... with these stats...
results_types = [
    '_(median)',
    '_(low_5%)',
    '_(high_95%)',
    '_(mean)',
    '_(stdev)',
    '_(95ci)',
]
# ... and gather all combinations of measure and stat here:
results_columns = []
for column in outcome_results_columns:
    columns_here = [column + ending for ending in results_types]
    results_columns += columns_here
# Also store onset to needle time:
# results_columns += ['Onset_to_needle_(mean)']


results_df = pd.DataFrame(columns=results_columns)

# trial dataframe is set up each scenario, but define column names here
trial_columns = [
    'Thrombolysis_rate_(%)',
    'LVO_IVT_mean_shift',
    'LVO_MT_mean_shift',
    'nLVO_IVT_mean_shift',
    # 'onset_to_needle'
    ]

# Iterate through hospitals
for hospital in hospital_performance.iterrows():
    scenario_counter += 1
    print(f'Scenario {scenario_counter}', end='\r')

    # Get data for one hospital
    hospital_name = hospital[0]
    hospital_data = hospital[1]
    run_data = hospital_data
    calibration = 1.0

    # Set up trial results dataframe
    trial_df = pd.DataFrame(columns=trial_columns)

    number_of_patients = int(run_data['admissions'])

    for trial in range(trials):

        print('start', end='\r')
        patient_array = SSNAP_Pathway(hospital_name, hospital_data)
        patient_array.run_trial()
        print('end', end='\r')
        
        # Initiate the outcome model object:
        clinical_outcome = Clinical_outcome(mrs_dists, number_of_patients)
        # Import patient array data:
        for key in clinical_outcome.trial.keys():
            if key in patient_array.trial.keys():
                clinical_outcome.trial[key].data = patient_array.trial[key].data
        
        # Calculate outcomes:
        results, combo_results = clinical_outcome.calculate_outcomes()
        
#         # mask = (
#         #     patient_array.patient_array_thrombolysis_conditions_met_bool == 1)
#         # onset_to_needle_mins_masked = \
#         #         patient_array.patient_array_onset_to_needle_mins[mask].mean()


        # Save scenario results to dataframe
        result = [
            np.mean(patient_array.trial['ivt_chosen_bool'].data)*100.0,
            results['lvo_ivt_mean_valid_patients_mean_mrs_shift'],
            results['lvo_mt_mean_valid_patients_mean_mrs_shift'],
            results['nlvo_ivt_mean_valid_patients_mean_mrs_shift'],
            # onset_to_needle_mins_masked
        ]
        # print('result', result)
        trial_df.loc[trial] = result
        
        # print(stop, here, please)
        
    trial_result = []
    
    for column in outcome_results_columns:
        results_here = [
            trial_df[column].median(),
            trial_df[column].quantile(0.05),
            trial_df[column].quantile(0.95),
            trial_df[column].mean(),
            trial_df[column].std(),
            (trial_df[column].mean() -
                stats.norm.interval(0.95, loc=trial_df[column].mean(),
                scale=trial_df[column].std() / sqrt(trials))[0]),
        ]
        trial_result += results_here
    # trial_result += [trial_df['onset_to_needle'].mean()]
    
    # add scenario results to results dataframe
    results_df.loc[hospital_name] = trial_result
    

# Apply calibration
results_df['calibration'] = calibration
for col in list(results_df):
    if 'Percent_Thrombolysis' in col or 'Additional_good_outcomes' in col:
        results_df[col] *= calibration

# round all results to 2 decimal places and return
# results_df = results_df.round(2)
# return (results_df)


/home/anna/miniconda3/lib/python3.9/site-packages/scipy/stats/_distn_infrastructure.py:2351: RuntimeWarning: invalid value encountered in multiply
  lower_bound = _a * scale + loc
/home/anna/miniconda3/lib/python3.9/site-packages/scipy/stats/_distn_infrastructure.py:2352: RuntimeWarning: invalid value encountered in multiply
  upper_bound = _b * scale + loc


/home/anna/miniconda3/lib/python3.9/site-packages/scipy/stats/_distn_infrastructure.py:2351: RuntimeWarning: invalid value encountered in multiply
  lower_bound = _a * scale + loc
/home/anna/miniconda3/lib/python3.9/site-packages/scipy/stats/_distn_infrastructure.py:2352: RuntimeWarning: invalid value encountered in multiply
  upper_bound = _b * scale + loc


/home/anna/miniconda3/lib/python3.9/site-packages/scipy/stats/_distn_infrastructure.py:2351: RuntimeWarning: invalid value encountered in multiply
  lower_bound = _a * scale + loc
/home/anna/miniconda3/lib/python3.9/site-packages/scipy/stats/_distn_infrastructure.py:2352: RuntimeWarning: invalid value encountered in multiply
  upper_bound = _b * scale + loc


/home/anna/miniconda3/lib/python3.9/site-packages/scipy/stats/_distn_infrastructure.py:2351: RuntimeWarning: invalid value encountered in multiply
  lower_bound = _a * scale + loc
/home/anna/miniconda3/lib/python3.9/site-packages/scipy/stats/_distn_infrastructure.py:2352: RuntimeWarning: invalid value encountered in multiply
  upper_bound = _b * scale + loc


/home/anna/miniconda3/lib/python3.9/site-packages/scipy/stats/_distn_infrastructure.py:2351: RuntimeWarning: invalid value encountered in multiply
  lower_bound = _a * scale + loc
/home/anna/miniconda3/lib/python3.9/site-packages/scipy/stats/_distn_infrastructure.py:2352: RuntimeWarning: invalid value encountered in multiply
  upper_bound = _b * scale + loc


n.b. runtime warning about multiply error a * scale + loc is something to do with the above cell, not either of the pathway or outcome classes it's calling. Maybe that stats.norm.interval() line at the end?

In [ ]:
hospital_data.to_dict()

In [ ]:
patient_array.trial['onset_to_arrival_mins'].data

In [ ]:
patient_array.trial['onset_time_known_bool'].data == 0

In [ ]:
repr(patient_array)

In [ ]:
print(patient_array)

In [16]:
results_df.head(5).T

,0,1,2,3,4
Thrombolysis_rate_(%)_(median),15.282392,13.477366,8.762887,10.840708,14.511041
Thrombolysis_rate_(%)_(low_5%),13.122924,10.905350,7.206186,8.174779,12.452681
Thrombolysis_rate_(%)_(high_95%),17.774086,15.648148,10.742268,13.274336,16.719243
Thrombolysis_rate_(%)_(mean),15.368771,13.370370,8.898969,10.732301,14.498423
Thrombolysis_rate_(%)_(stdev),1.435279,1.428547,1.161832,1.539706,1.407501
Thrombolysis_rate_(%)_(95ci),0.281310,0.279990,0.227715,0.301777,0.275865
LVO_IVT_mean_shift_(median),0.001921,0.001819,0.001044,0.001487,0.002304
LVO_IVT_mean_shift_(low_5%),0.001395,0.001321,0.000683,0.001031,0.001796
LVO_IVT_mean_shift_(high_95%),0.002389,0.002300,0.001443,0.001956,0.002659
LVO_IVT_mean_shift_(mean),0.001930,0.001834,0.001057,0.001483,0.002273


Print some info about the final trial:

In [ ]:
print(clinical_outcome)

In [ ]:
patient_array.trial.keys()

In [ ]:
results_df

In [ ]:
results_df.to_csv('full_outcome_pathway_results_refactored.csv')